# Analyse des ventes
---
<div class="alert alert-block alert-info">
<b>🗣</b>Ce projet est un exemple dont le but est de vous aidez à envisager les problèmes liés à votre entreprise sous l'angle de l'analyse de vos données. Notre objectif  est de vous démontrer que l'extraction de connaissances à partir de vos données facilite les prises de décisions et constitue un avantage stratégique.
</div>

## Sommaire
Dans cet exemple __nous analyserons les ventes d'une entreprise fictif en ligne d’électronique localisé au US__
1. [Présentation des données](#load)
2. [Exploration et préparation des données](#cleanning)
3. [Augment Data with Additional Column](#augment)
4. [Questions](#question)

In [180]:
import os
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
np.random.seed(42)

## 1. Présentation des données 
---
Les données utilisées sont collectéess àpres chaque vente auprès des clients de notre entreprise fictif en ligne d’électronique. Lors d'un achat, on collecte et on sauvegarde des informations sur le client dans une base de données. Cette base de données forme le départ de notre analyse car elle contient des informations pertinentes sur nos clients, tel que les _produits achetés_ par ce dernier, _l'heure d'achat_ où encore _l’adresse de livraison_. En outre, __définir l'ensemble des informations appropriées à collecter sur nos clients afin de mener à bien une analyse de qualité est une étape indispensable dans le processus d’analyse des données__. Voici-ci dessous un descriptif des données collecter pour chaque client dans le cadre de notre exemple.

In [181]:
data = pd.read_csv('all_data.csv')
data.head()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
0,295665,Macbook Pro Laptop,1,1700,12/30/19 00:01,"136 Church St, New York City, NY 10001"
1,295666,LG Washing Machine,1,600.0,12/29/19 07:03,"562 2nd St, New York City, NY 10001"
2,295667,USB-C Charging Cable,1,11.95,12/12/19 18:21,"277 Main St, New York City, NY 10001"
3,295668,27in FHD Monitor,1,149.99,12/22/19 15:13,"410 6th St, San Francisco, CA 94016"
4,295669,USB-C Charging Cable,1,11.95,12/18/19 12:38,"43 Hill St, Atlanta, GA 30301"


↳ Le client répertorié par l'id __295665__ a acheté un __Macbook Pro__  à __1700$__ le __30 décembre 2019 à 00:01__, son adress de livraison est __136 Church St, New York City, NY 10001__ 

### Information Complémentaire sur les données collectées
- __ID__, numéro unique définissant le client 
- __Produit__, nom du matériel informatique acheté 
- __Quantité commandée__, nombre d’exemplaires vendu
- __Prix__, prix unnitaire de chaque produit
- __Date__, date et heure de l'achat
- __Adresse__, adresse de livraison

⚠️ Si la commande d'un client contient plusieur produit différents alors chaque produit sera répertorié sur sa propre ligne. Voir exemple ci-dessous.

In [182]:
data[data[['Order ID','Order Date']].duplicated()][:3]

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
17,295681,USB-C Charging Cable,1,11.95,12/25/19 12:37,"79 Elm St, Boston, MA 02215"
18,295681,Bose SoundSport Headphones,1,99.99,12/25/19 12:37,"79 Elm St, Boston, MA 02215"
19,295681,Wired Headphones,1,11.99,12/25/19 12:37,"79 Elm St, Boston, MA 02215"


↳ le client avec l'id 295681 a commandé USB-C Charging Cable, Wired Headphones, Bose SoundSport Headphones 

Après cette présentation rapide des données et avant de visualiser et d'analyser les ventes en ligne, __il est nécessaire d'explorer et de préparer notre jeu de données__. Cet étape bien que chronophage est indispensable pour d'obtenir une analyse de qualité. Cependant elle apporte peut d’intérêt pour un lecteur. Si vous ne voulez pas rentrer dans les détails et passer à la section suivante [cliquez ici]()

# 2. Explorartion et préparation des données
---
<a id=cleanning></a>
L’apparition de données manqunates ou erronées est quasi innevitable lors de la collecte.
__Il est donc essentiel d'explorer notre jeu de données afin de repérer et filtrer les données inutilisables__.  

In [183]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186850 entries, 0 to 186849
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Order ID          186305 non-null  object
 1   Product           186305 non-null  object
 2   Quantity Ordered  186305 non-null  object
 3   Price Each        186305 non-null  object
 4   Order Date        186305 non-null  object
 5   Purchase Address  186305 non-null  object
dtypes: object(6)
memory usage: 8.6+ MB


A l'aide des informations ci-dessus, nous pouvons remarquer que sur les 186 850 lignes de notre base de données , __nous avons un total de 186 305 ventes, le reste (soit 545 lignes) correspond à des lignes entières de données manquantes (NaN)__. Voir la table ci-dessous

In [184]:
data[data.isnull().any(1)] # affiche les lignes de données manquantes 

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
264,NaN,NaN,NaN,NaN,NaN,NaN
648,NaN,NaN,NaN,NaN,NaN,NaN
680,NaN,NaN,NaN,NaN,NaN,NaN
1385,NaN,NaN,NaN,NaN,NaN,NaN
1495,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
185795,NaN,NaN,NaN,NaN,NaN,NaN
185868,NaN,NaN,NaN,NaN,NaN,NaN
185887,NaN,NaN,NaN,NaN,NaN,NaN
185960,NaN,NaN,NaN,NaN,NaN,NaN


Ces lignes entières de données manquantes _(NaN)_ n'ont pas d'intérêt et bloquerons notre marge de manœuvre dans les prochaines étapes de notre analyse. __Il faut les supprimer!__

In [185]:
data.dropna(how='all',inplace=True) # supprime les lignes manquantes
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 186305 entries, 0 to 186849
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Order ID          186305 non-null  object
 1   Product           186305 non-null  object
 2   Quantity Ordered  186305 non-null  object
 3   Price Each        186305 non-null  object
 4   Order Date        186305 non-null  object
 5   Purchase Address  186305 non-null  object
dtypes: object(6)
memory usage: 9.9+ MB


Et voila, on a un total de 186 305 lignes pour 186 305 données non null - pour chaque colonne. __Il n'y a plus de données manquantes _(NaN)_ dans  notre tableau__

⚠️ Cependant après investigation, __on remarque qu'il y a d'autre données erronées dans notre tableau__.

In [186]:
data[data['Order Date'].str.contains('Or')]

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
254,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
705,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
1101,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
2875,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
3708,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
...,...,...,...,...,...,...
183671,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
184012,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
184041,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
184275,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address


↳ Il semblerait que certaines des lignes collectées sont une simple copie du nom des collonnes du tableau. Supprimons les!

In [187]:
data = data[data['Order Date'].str[:2] != 'Or']

In [188]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 185950 entries, 0 to 186849
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Order ID          185950 non-null  object
 1   Product           185950 non-null  object
 2   Quantity Ordered  185950 non-null  object
 3   Price Each        185950 non-null  object
 4   Order Date        185950 non-null  object
 5   Purchase Address  185950 non-null  object
dtypes: object(6)
memory usage: 9.9+ MB


__Maintenant qu'il n'y a plus de données erronées, nous pouvons convertir chaque collones au format qui lui est le plus adaptée__. Pour le moment chaque collone est au format `object`. A l'aide du code ci-dessous nous allons transformer le format de 3 collones:
- les __Quantité commandée__ en `entier` 
- la __Date__ en `datetime`
- le __Prix unitaire__ en `float`

In [189]:
data['Price Each'] = data['Price Each'].astype('float32')
data['Quantity Ordered'] = data['Quantity Ordered'].astype('int32')
data.loc[:,'Order Date'] = pd.to_datetime(data['Order Date'])

In [190]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 185950 entries, 0 to 186849
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   Order ID          185950 non-null  object        
 1   Product           185950 non-null  object        
 2   Quantity Ordered  185950 non-null  int32         
 3   Price Each        185950 non-null  float32       
 4   Order Date        185950 non-null  datetime64[ns]
 5   Purchase Address  185950 non-null  object        
dtypes: datetime64[ns](1), float32(1), int32(1), object(3)
memory usage: 8.5+ MB


__↳ Et voilà, toutes les données erronées ont été filtrées et les valeurs de chaque colonne ont été converties dans leurs format le plus adaptées! 🥳__  

# 3. Création de nouveau paramètres <a id=augment><a/>
---

Bien que les données soient dans un format optimal pour leur exploitation, __nous allons dédié la section suivante à la creation de nouveaux paramètres__.  
    
Souvent créés à partir de collones déjà existante nous essayons de créer des paramètres rendant notre etude plus pértinante. Cette phase souvent appelé _feature engineering_ est une étape récursive qui est completé lorsqu'on veut tester la pertinance de nouveau paramètre dans notre analyse. 

### Mois et Heure

In [191]:
data['Hour'] = data['Order Date'].dt.hour
data.loc[:,'Month_num'] = data['Order Date'].dt.month
month_list = ['Janvier','Février','Mars','Avril',
         'Mai','Juin','Juillet','Août', 'Septembre','Octobre','Novembre', 'Décembre']
data['Month'] =  data['Month_num'].apply(lambda x: month_list[x-1])
data.head()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address,Hour,Month_num,Month
0,295665,Macbook Pro Laptop,1,1700.000000,2019-12-30 00:01:00,"136 Church St, New York City, NY 10001",0,12,Décembre
1,295666,LG Washing Machine,1,600.000000,2019-12-29 07:03:00,"562 2nd St, New York City, NY 10001",7,12,Décembre
2,295667,USB-C Charging Cable,1,11.950000,2019-12-12 18:21:00,"277 Main St, New York City, NY 10001",18,12,Décembre
3,295668,27in FHD Monitor,1,149.990005,2019-12-22 15:13:00,"410 6th St, San Francisco, CA 94016",15,12,Décembre
4,295669,USB-C Charging Cable,1,11.950000,2019-12-18 12:38:00,"43 Hill St, Atlanta, GA 30301",12,12,Décembre


### Prix global

In [192]:
data['Sales'] = data['Price Each'] * data['Quantity Ordered']
data.head()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address,Hour,Month_num,Month,Sales
0,295665,Macbook Pro Laptop,1,1700.000000,2019-12-30 00:01:00,"136 Church St, New York City, NY 10001",0,12,Décembre,1700.000000
1,295666,LG Washing Machine,1,600.000000,2019-12-29 07:03:00,"562 2nd St, New York City, NY 10001",7,12,Décembre,600.000000
2,295667,USB-C Charging Cable,1,11.950000,2019-12-12 18:21:00,"277 Main St, New York City, NY 10001",18,12,Décembre,11.950000
3,295668,27in FHD Monitor,1,149.990005,2019-12-22 15:13:00,"410 6th St, San Francisco, CA 94016",15,12,Décembre,149.990005
4,295669,USB-C Charging Cable,1,11.950000,2019-12-18 12:38:00,"43 Hill St, Atlanta, GA 30301",12,12,Décembre,11.950000


### Ville et État

In [193]:
def get_city(adress):
    return adress.split(',')[1]

def get_state(adress):
    return adress.split(',')[2].split(' ')[1]

data['City'] = data['Purchase Address'].apply(lambda x:f'{get_city(x)} ({get_state(x)})')
data.head()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address,Hour,Month_num,Month,Sales,City
0,295665,Macbook Pro Laptop,1,1700.000000,2019-12-30 00:01:00,"136 Church St, New York City, NY 10001",0,12,Décembre,1700.000000,New York City (NY)
1,295666,LG Washing Machine,1,600.000000,2019-12-29 07:03:00,"562 2nd St, New York City, NY 10001",7,12,Décembre,600.000000,New York City (NY)
2,295667,USB-C Charging Cable,1,11.950000,2019-12-12 18:21:00,"277 Main St, New York City, NY 10001",18,12,Décembre,11.950000,New York City (NY)
3,295668,27in FHD Monitor,1,149.990005,2019-12-22 15:13:00,"410 6th St, San Francisco, CA 94016",15,12,Décembre,149.990005,San Francisco (CA)
4,295669,USB-C Charging Cable,1,11.950000,2019-12-18 12:38:00,"43 Hill St, Atlanta, GA 30301",12,12,Décembre,11.950000,Atlanta (GA)


### Latitude et Longitude

In [217]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="rasseax@gmail.com")
lat_long = {city:(geolocator.geocode(city).latitude,geolocator.geocode(city).longitude) 
            for city in data['City'].unique()}

In [218]:
data['lat'] = data['City'].apply(lambda city: lat_long[city][0])
data['long'] = data['City'].apply(lambda city: lat_long[city][1])

---
# 4. Questions
<a id=question><a/>

## Quel a été le meilleur mois de vente ? Combien a-t-on gagné ce mois-là ?

In [194]:
sale_per_month = data.groupby(['Month_num', 'Month'])['Sales'].sum()
sale_per_month.sort_values(ascending=False)

Month_num  Month    
12         Décembre     4.613443e+06
10         Octobre      3.736727e+06
4          Avril        3.390670e+06
11         Novembre     3.199603e+06
5          Mai          3.152607e+06
3          Mars         2.807100e+06
7          Juillet      2.647776e+06
6          Juin         2.577802e+06
8          Août         2.244468e+06
2          Février      2.202022e+06
9          Septembre    2.097560e+06
1          Janvier      1.822257e+06
Name: Sales, dtype: float64

↳ Les ventes du mois de décembre sont le meilleurs avec des ventes à 4.61 M de dollars il surpasse de 15% le 2eme mois le plus prolifique, Octobre. Quant au mois le moins rentable il s'agit du mois de Janvier avec 1.8 M$ de ventes soit 2XX % moins que le mois de decembre 

In [195]:
best_sale = go.Figure(go.Bar(x = sale_per_month.index.get_level_values(1), y = sale_per_month))
best_sale.update_layout(showlegend=False, hovermode="x unified", title = "Total des ventes par mois")
best_sale.update_yaxes(title= 'Ventes en  USD($)')
best_sale.update_xaxes(title= None)
best_sale.show()

---
## Quelle est la ville qui a réalisé le plus grand nombre de ventes ?

In [196]:
city_sales = data.groupby('City').sum()['Sales']
city_sales

City
 Atlanta (GA)          2.795499e+06
 Austin (TX)           1.819582e+06
 Boston (MA)           3.661642e+06
 Dallas (TX)           2.767975e+06
 Los Angeles (CA)      5.452571e+06
 New York City (NY)    4.664317e+06
 Portland (ME)         4.497583e+05
 Portland (OR)         1.870732e+06
 San Francisco (CA)    8.262204e+06
 Seattle (WA)          2.747755e+06
Name: Sales, dtype: float64

In [210]:
best_city = go.Figure(go.Bar(x = city_sales.index, y = city_sales))
best_city.update_layout(showlegend=False, hovermode="x unified", 
                        title = "Total des ventes durant l’année par ville")
best_city.update_yaxes(title= 'Ventes en  USD($)')
best_city.update_xaxes(title= 'Ventes par ville', showticklabels=False)
best_city.show()

### vision geographique

In [223]:
data['State'] = data['Purchase Address'].apply(lambda x:f'{get_state(x)}')

# get acess to mapbox
with open('mapbox_token.txt') as f:
    lines=[x.rstrip() for x in f]
    
mapbox_access_token = lines[0]
px.set_mapbox_access_token(mapbox_access_token)

In [229]:
test = data.groupby(['City','lat','long']).sum()['Sales'].reset_index()
test

,City,lat,long,Sales
0,Atlanta (GA),33.749099,-84.390185,2.795499e+06
1,Austin (TX),30.271129,-97.743700,1.819582e+06
2,Boston (MA),42.360253,-71.058291,3.661642e+06
3,Dallas (TX),32.776272,-96.796856,2.767975e+06
4,Los Angeles (CA),34.053691,-118.242767,5.452571e+06
5,New York City (NY),40.712728,-74.006015,4.664317e+06
6,Portland (ME),29.065489,-110.971705,4.497583e+05
7,Portland (OR),45.520247,-122.674195,1.870732e+06
8,San Francisco (CA),46.844325,-71.274327,8.262204e+06
9,Seattle (WA),47.603832,-122.330062,2.747755e+06


In [239]:
fig = px.scatter_mapbox(test, lat="lat", lon="long", size="Sales", zoom=3)
fig.show()

ne semble pas être super précis 

In [240]:
# ne marche pas
map_plot = go.Figure(go.Scattermapbox(
        lat = test['lat'], 
        lon = test['long'],
        marker = go.scattermapbox.Marker(size= test['Sales']),
                        ))
map_plot.show()

---
## What time we should display advertisement to maximize likelihood of customer's buying product?

In [ ]:
buying_hours = data.groupby('Hour').sum()['Quantity Ordered']
buying_hours.sort_values(ascending=False)

In [ ]:
plt.figure(figsize=(10,8))
buying_hours.plot.bar()
plt.title('Quantity of purchase per hours', fontsize=20)
plt.ylabel('Quantity ordered', fontsize=16)
plt.xlabel('Hour', fontsize=16)
plt.xticks(rotation= 0)
save_fig('quantity_ordered_per_hours')
plt.show()

---
## What products are more often solde together?

In [ ]:
# drop the command of only one product
multi_purchase = data[data['Order ID'].duplicated(keep=False)]

In [ ]:
# gather in one cell all article purchase by 'Order ID'
multi_purchase.loc[:,'Grouped'] = multi_purchase.groupby('Order ID')['Product'].transform(lambda x: ','.join(x))

In [ ]:
# drop the duplicate ID
multi_purchase = multi_purchase.drop_duplicates(subset='Order ID')
multi_purchase.head()

In [ ]:
# Count the number of combination 
from itertools import combinations
from collections import Counter

count = Counter()

for row in multi_purchase['Grouped']:
    row_list = row.split(',')
    count.update(Counter(combinations(row_list, 2)))

for key,value in count.most_common(10):
    print(key, value)

 ↳ Referenced: [stackoverflow](https://stackoverflow.com/questions/52195887/counting-unique-pairs-of-numbers-into-a-python-dictionary)

---
### What product sold the most? Why do you think it sold the most?

In [ ]:
product_sales = data.groupby('Product').sum()['Quantity Ordered']
product_sales

In [ ]:
plt.figure(figsize=(10,8))
product_sales.plot.bar()
plt.title('The product sold the most', fontsize=20)
plt.ylabel('Quantity Ordered', fontsize=16)
save_fig('product_sold_the_most')
plt.show()

In [ ]:
prices = data.groupby('Product').mean()['Price Each']

In [ ]:
fig, ax1 = plt.subplots(figsize=(10,8))
ax2 = ax1.twinx()
ax2.plot(prices.index, prices, color='r')
ax1.bar(product_sales.index, product_sales, color='b',alpha=0.6)

ax1.set_ylabel('Quantity Ordered', color='b')
ax2.set_ylabel('Price ($)', color='r')
ax1.set_xticklabels(prices.index, rotation='vertical', size=8)
save_fig('Most_saled_product')